In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import importlib
import plotly.express as px
from datetime import datetime
from datastory import DataStory

sys.path.append('/home/jupyter/pensjon-data-analyse/lib')
import utils
import pandas_utils
import pesys_utils

importlib.reload(utils)
importlib.reload(pandas_utils)
importlib.reload(pesys_utils)

utils.set_secrets_as_env()

In [ ]:
con = pesys_utils.open_pen_connection()

df_levetid = pandas_utils.pandas_from_sql('/home/jupyter/pensjon-data-analyse/sql/levetid_krav.sql', con, tuning=1000)
con.close()
df_levetid.columns = map(str.lower, df_levetid.columns)
df_levetid["dato"] = datetime.utcnow()

In [ ]:
df = df_levetid.copy()
df.head()

In [ ]:
df["dato_opprettet"] = df.dato_opprettet.apply(pd.Timestamp.to_pydatetime)
df["dato_endret"] = df.dato_endret.apply(pd.Timestamp.to_pydatetime)

In [ ]:
df["levetid"] = df.dato.subtract(df.dato_opprettet).dt.days
df["uendret"] = df.dato.subtract(df.dato_endret).dt.days

In [ ]:
df.head()

In [ ]:
px.histogram(df, "levetid")

In [ ]:
px.histogram(df, "uendret")

In [ ]:
df_krav_over_ett_ar = df[df.uendret > 365]
anke_filter =  ~((df_krav_over_ett_ar.sakstype == "Uføretrygd") & (df_krav_over_ett_ar.kravtype == "Anke"))
df_sunburst = df_krav_over_ett_ar[anke_filter]

In [ ]:
fig_sunburst = px.sunburst(df_sunburst, path=["sakstype", "kravtype", "kravstatus"], color_discrete_sequence=px.colors.qualitative.Dark2, height=1000)
fig_sunburst.show()

In [ ]:
ds = DataStory('"Glemte krav" i pesys')
ds.markdown("Oversikt over krav i pesys hvor 'endret_dato' er over et år siden, og status ikke er 'FERDIG' eller 'AVBRUTT'. Anker på Uføretrygd er ikke inkludert.")
ds.plotly(fig_sunburst.to_json())
#ds.publish(url="https://nada.intern.nav.no/api")